In [12]:
import os
import urllib
import pandas as pd
import random
from statsmodels.robust import scale
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve


In [13]:
df = pd.read_csv('/Users/naina/Documents/Personal/Coding/ebay_auctions.csv')

In [14]:
df

,item,days,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
0,xbox,7,8211480551,52.99,1.201505,hanna1104,94.0,49.99,311.60
1,xbox,7,8211480551,50.99,1.203843,wrufai1,90.0,49.99,311.60
2,xbox,7,8211480551,101.99,1.204433,wrufai1,90.0,49.99,311.60
3,xbox,7,8211480551,57.00,1.708437,newberryhwt,14.0,49.99,311.60
4,xbox,7,8211480551,144.48,3.089711,miloo2005,3.0,49.99,311.60
5,xbox,7,8211480551,170.00,4.187963,fzakhour,5.0,49.99,311.60
6,xbox,7,8211480551,189.00,5.245810,bbcboi2007,1.0,49.99,311.60
7,xbox,7,8211480551,298.48,5.863021,miloo2005,3.0,49.99,311.60
8,xbox,7,8211480551,200.00,6.334815,laxwarrior07,12.0,49.99,311.60
9,xbox,7,8211480551,250.00,6.548796,aladdin987,292.0,49.99,311.60


In [15]:
def difference(x, y):
    return abs(x - y)

df['delta'] = difference(df['price'], df['openbid'])

In [16]:
byid = df.groupby('auctionid')

In [17]:
stats = {
    'bid': {
        'std_dev_bids': 'std',
        'average_bids': 'mean',
        'total_bids': 'count'
        },
    'delta': {
        'delta': 'mean'
    }
}

In [8]:
new_df = byid.agg(stats)
new_df.columns = new_df.columns.droplevel(level=0)
new_df

,average_bids,total_bids,std_dev_bids,delta
auctionid,,,,
1638843936,1167.857143,14,489.252626,1125.00
1638844284,362.500000,4,158.771324,300.00
1638844464,554.236250,32,130.438535,440.00
1638844729,284.545455,22,29.797295,95.00
1638893549,144.500000,5,34.022052,78.50
1638917885,135.079231,26,70.624805,202.50
1639226378,1057.083333,24,389.372172,1230.00
1639253454,252.500000,4,2.886751,5.00
1639309309,191.575556,54,86.235150,373.99


In [9]:
train, test = train_test_split(new_df, test_size = 0.6)
train

,average_bids,total_bids,std_dev_bids,delta
auctionid,,,,
3015344188,233.750000,2,1.767767,5.00
3022018351,225.000000,1,NaN,0.00
1649848613,474.950000,40,376.946636,1000.00
3020125049,207.448000,10,15.924939,53.49
8214784050,23.553333,9,8.266565,21.01
8214864154,71.730000,12,12.089642,34.88
3024428241,186.040000,6,8.653776,22.62
3024799631,180.596111,18,53.959940,155.01
8214435010,42.800571,35,43.040301,121.51


In [10]:
import statsmodels.formula.api as sm
result = sm.ols(formula="delta ~ average_bids + std_dev_bids", data=test).fit()
print(result.params)

Intercept      -0.224271
average_bids    0.066683
std_dev_bids    3.070601
dtype: float64


In [31]:
print(result.params[0])
print(result.params[1])
print(result.params[2])

-0.224271487033
0.0666829216248
3.07060107093


In [29]:
def calculatedelta(average, stddev):
    delta = result.params[0] + result.params[1]*average + result.params[2]*stddev
    return delta